# Calculate Q-Rookley

- Input data "20172022_processed_1_3_4.csv", from 2017/7/1 to 2022/12/31, same as used in Figlewski

- Use 500 rolling window for P density (when calculating EPK)

- Use logreturn instead of log+1

- Use 
```
r.source('Q_Rookley_main_3.R')
```
and
```
'EPK_library_1.r'
```
to calculate Q_Rookley, along with 
```
get_btc_prices_2015_2022
```
for physical density.

- Interest rate: 0

- Moneyness range is [min moneyness, max moneyness]

- Bandwidth 0.3

- tau=13

In [1]:
import platform
plat = platform.system()
import locale
if plat == 'Darwin':
    print('Using Macos Locale')
    locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
else:
    locale.setlocale(locale.LC_ALL, 'en_US.utf8')
import datetime
import pandas as pd
import sys
import matplotlib.pyplot as plt
import csv
import gc
import math
import numpy as np
import os
import pdb
import pickle
import statsmodels.api as sm
import time


import rpy2.robjects.packages as rpackages
from rpy2 import robjects

from scipy import stats
from scipy.integrate import simps
from scipy import interpolate

from src.helpers import save_dict, load_dict
from src.brc import BRC
from src.strategies import IronCondor
from src.quandl_data import get_btc_prices_2015_2022

from numpy.lib.function_base import insert

from statsmodels.nonparametric.kernel_regression import KernelReg as nadarajawatson

from sklearn.neighbors import KernelDensity

from pathlib import Path

def filter_sub(_df,  mindate, maxdate, tau):
    # Subset

    # Only calls, tau in [0, 0.25] and fix one day (bc looking at intra day here)
    # (_df['is_call'] == 1) & --> Dont need to filter for calls because that happens in the R script
    # Also need to consider put-call parity there
    sub = _df[(_df['date'] >= mindate) & (_df['date'] <= maxdate) & (_df['moneyness'] >= 0.5) & (_df['moneyness'] < 2) &(_df['mark_iv'] > 0)]# &

    if tau > 0:
        sub = sub[sub['tau'] == tau]

    nrows = sub.shape[0]
    if nrows == 0:
        raise(ValueError('Sub is empty'))
   
    sub['moneyness'] = round(sub['moneyness'], 3)
    sub['underlying_price'] = round(sub['underlying_price'], 2)

    sub = sub.drop_duplicates()
    
    print(sub.describe())

    if nrows > 50000:
        print('large df, subsetting')
        sub = sub.sample(10000)
        print(sub.describe())

    return sub

def prepare_confidence_band_data(df):

            # Old
                # Save for Confidence Band 
                    # Column 1 - Day
                    # Column 2 - Month
                    # Column 3 - Year
                    # Column 4 - Call Indicator
                    # Column 5 - Time Maturity in Days
                    # Column 6 - Strike
                    # Column 7 - IV
                    # Column 8 - Spot
                    # Column 9 - Risk Free Rate
                    # Column 10 - 
                    # Column 11 - Forward
                # 3	1	1997	0	49	3025	165.8	2861.706	0.031465	0.150026951	2863.26
                # Day Month Year Put Days Strike IV     Spot         Risk Free    Tau in Days (255)       Forward

    """
    # Mapping Table for Input

    # Column ... X
    # 1 - date
    # 2 - IV
    # 3 - 1 for Call, 0 for Put
    # 4 - Tau 
    # 5 - Strike
    # 6 - assigned meanwhile as put2call[,6]+ put2call[,7] -  put2call[,5]*exp(- mean(put2call[,8])* tau);
    # 7 - Spot
    # 8 - Probably Interest Rate
    # 9 - assigned meanwhile Moneyness: Spot/Strike
    """
    b = df[['date', 'mark_iv', 'putcall', 'tau', 'K', 'underlying_price', 'interest_rate', 'moneyness', 'instrument_price']]
    b['placeholder'] = 0
    b = b.reset_index(drop=True)
    # b['interest_rate'] = 0.00
    b['forward'] = 1

    # Reorder
    df_c = b[['date', 'mark_iv', 'putcall', 'tau', 'K', 'instrument_price', 'underlying_price', 'interest_rate', 'moneyness']]

    unique_maturities = df.days_to_maturity.unique()[0]
    fname = str('confidence_band_input_') + str(unique_maturities) + str('.csv')
    df_c.to_csv(fname, index = False)
    return fname

def bootstrap(conf_fname, phys_fname, rdate, tau, simmethod, r, out_dir, bw, lower, upper):
                            
    try:
        # MAKE SURE TO USE DIFFERENT 2ND INPUT FOR SP500!!
        moneyness, spd, epk, epk_lo, epk_up, logret, spd_logret, epk_logret, epk_logret_lo, epk_logret_up, epk_BS, tau, volas, spd_lo, spd_up, cdf_m, cdf_ret= r.bootstrap_epk(robjects.StrVector([conf_fname]), 
                                      robjects.StrVector([phys_fname]),
                                      robjects.StrVector(rdate),
                                      robjects.StrVector([out_dir]),
                                      robjects.FloatVector([bw]),
                                      robjects.FloatVector([lower]),
                                      robjects.FloatVector([upper])
                                     )
        # Can recover PD here as SPD/EPK
        spd_df = pd.DataFrame({'m': moneyness,
                               'spdy': spd,
                               'spdy_up': spd_up,
                               'spdy_dn': spd_lo,
                               'epk': epk,
                               'ret': logret,
                               'spd_ret': spd_logret,
                               'cb_ret_up': epk_logret_up,
                               'cb_ret_dn': epk_logret_lo,
                               'epk_ret': epk_logret,
                               'volatility': volas,
                               'epk_up': epk_up,
                               'epk_dn': epk_lo,
                               'cdf_m': cdf_m,
                               'cdf_ret': cdf_ret})

        return spd_df
    
    except Exception as e:
        print('Sim failed, proceeding with the next one... - ', simmethod)
        print('exception: ', e)
        
# Initiate R Objects
base = rpackages.importr("base") 
r = robjects.r
r.source('src/confidence_bands.R')
r.source('src/Q_Rookley_main_3.R')

# BTC prices
tdat = get_btc_prices_2015_2022()
tdat[::-1].to_csv('Data/BTC_USD_Quandl_2015_2022.csv', index = False)

bw_range = np.array([0.3])
r_bandwidth = bw_range[0]

simmethod = ['oldschool']

df = pd.read_csv('Data/20172022_processed_1_3_4.csv')
interest_rate_data = pd.read_csv('Data/IR_daily.csv')
interest_rate_data = interest_rate_data.rename(columns={'index': 'date','DTB3': 'interest_rate'})

df1=df[df['tau']==13]
dates=df1['date'].unique()
#df1=df[df['date']>=dates[9]]
#dates = df1['date'].unique()

dates1 = dates

error_i = []
for i0 in range(dates1.size):
    df2 = df1[df1['date']==dates1[i0]]
    
    curr_day = datetime.datetime.strptime(dates1[i0],'%Y-%m-%d')
    curr_day_starttime = curr_day.replace(hour = 0, minute = 0, second = 0, microsecond = 0)
    curr_day_endtime = curr_day.replace(hour = 23, minute = 59, second = 59, microsecond = 0)
    print(curr_day)

    df3 = pd.merge(df2, interest_rate_data, on='date', how='left')
    

    df3['interest_rate'] = 0
    df3['days_to_maturity'] = df3['tau']
    df3['tau'] = df3['tau']/365
    df3['underlying_price'] = df3['BTC_price']
    df3['instrument_price'] = df3['option_price']
    df3['putcall'][df3['putcall']=='C']=1
    df3['putcall'][df3['putcall']=='P']=0
    
    df_1 = df3[['date','BTC_price', 'K', 'interest_rate', 'tau', 'putcall', 'IV',  'moneyness',
                'days_to_maturity',  'underlying_price', 'instrument_price']]
    df_1['mark_iv']=df_1['IV']/100
    
    df_1.to_csv('Results/option_' + datetime.datetime.strftime(curr_day_starttime, '%Y%m%d') + '.csv')
    
    conf_fname = prepare_confidence_band_data(df_1)
    
    rdate = base.as_Date(curr_day.strftime('%Y-%m-%d'))
    rdate_f = base.format(rdate, '%Y-%m-%d')
    
    tau = df['tau'].unique()[0]
    
    spd_btc = bootstrap(conf_fname, 'Data/BTC_USD_Quandl_2015_2022.csv', rdate_f, tau, simmethod, r, 
                                     'Results/Rookley_Q_CB_1722_1_2_5_1_ttm13', r_bandwidth, df_1['moneyness'].min(), df_1['moneyness'].max())
    
    try :
        
        spd_btc.to_csv('Results/Rookley_Q_CB_1722_1_2_5_1_ttm13/btc_pk_'  + str(curr_day_starttime.__format__('%Y-%m-%d')) +'_bw_' + str(r_bandwidth) +  '.csv')
        
        print(spd_btc)
        
    except Exception as e:
        error_i.append(i0)

Using Macos Locale


/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
R[write to console]: 
Attaching package: ‘matlab’


R[write to console]: The following object is masked from ‘package:stats’:

    reshape


R[write to console]: The following objects are masked from ‘package:utils’:

    find, fix


R[write to console]: The following object is masked from ‘package:base’:

    sum


R[write to console]: KernSmooth 2.23 loaded
Copyright M. P. Wand 1997-2009

2017-07-15 00:00:00


/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18265/1438358905.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18265/1438358905.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18265/1438358905.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] "bandwidth: 0.3"
[1] "test no arbitrage condition"
[1] 44.694713  4.722982 66.258718
                [,1]      [,2]         [,3]        [,4]
sigmas -0.2773002861 1.1774431 -1.839085776   6.9696769
       -0.2706737479 1.1683816 -1.806510876   6.9886556
       -0.2640908319 1.1594868 -1.774067534   7.0096857
       -0.2575509673 1.1507556 -1.741722168   7.0326339
       -0.2510535949 1.1421851 -1.709442458   7.0573672
       -0.2445981659 1.1337729 -1.677197359   7.0837526
       -0.2381841423 1.1255169 -1.644957103   7.1116577
       -0.2318109964 1.1174152 -1.612693205   7.1409504
       -0.2254782104 1.1094662 -1.580378467   7.1714990
       -0.2191852763 1.1016686 -1.547986978   7.2031724
       -0.2129316957 1.0940215 -1.515494119   7.2358402
       -0.2067169795 1.0865241 -1.482876564   7.2693723
       -0.2005406476 1.0791759 -1.450112277   7.3036399
       -0.1944022287 1.0719767 -1.417180513   7.3385145
       -0.1883012602 1.0649264 -1.384061814   7.3738688
       -0.18223

...

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]         [,3]     [,4]
sigmas -0.498850752 1.2412962 -2.879973175 5.280363
       -0.492414057 1.2300830 -2.859835187 5.295807
       -0.486018529 1.2189456 -2.839600133 5.310968
       -0.479663644 1.2078847 -2.819270814 5.325855
       -0.473348888 1.1969007 -2.798849952 5.340477
       -0.467073759 1.1859943 -2.778340195 5.354844
       -0.460837761 1.1751659 -2.757744118 5.368965
       -0.454640410 1.1644162 -2.737064216 5.382849
       -0.448481230 1.1537455 -2.716302912 5.396506
       -0.442359753 1.1431543 -2.695462550 5.409945
       -0.436275521 1.1326432 -2.674545399 5.423175
       -0.430228083 1.1222125 -2.653553651 5.436205
       -0.424216997 1.1118627 -2.632489422 5.449044
       -0.418241828 1.1015942 -2.611354752 5.461700
       -0.412302150 1.0914073 -2.590151604 5.474183
       -0.406397543 1.0813025 -2.568881865 5.486501
       -0.400527596 1.0712801 -2.547547348 5.498662
       -0.394691905 1.0613406 

        0.194749045 0.5704041  0.764952231 5.626024
        0.197971115 0.5735148  0.786406973 5.586056
        0.201182836 0.5767084  0.807631535 5.544807
        0.204384275 0.5799840  0.828617996 5.502260
        0.207575498 0.5833402  0.849358321 5.458400
        0.210756569 0.5867759  0.869844368 5.413210
        0.213927553 0.5902897  0.890067884 5.366675
        0.217088514 0.5938805  0.910020507 5.318779
        0.220239514 0.5975467  0.929693767 5.269506
        0.223380617 0.6012870  0.949079087 5.218839
        0.226511884 0.6050999  0.968167783 5.166765
        0.229633377 0.6089839  0.986951061 5.113266
        0.232745157 0.6129375  1.005420023 5.058327
        0.235847283 0.6169590  1.023565667 5.001934
        0.238939816 0.6210468  1.041378883 4.944070
        0.242022815 0.6251992  1.058850457 4.884720
        0.245096338 0.6294144  1.075971073 4.823869
        0.248160444 0.6336907  1.092731308 4.761503
        0.251215189 0.6380261  1.109121641 4.697605
        0.25

/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18265/1438358905.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='C']=1
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18265/1438358905.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['putcall'][df3['putcall']=='P']=0
/var/folders/_7/3860x3293tn4h2dlrp5_t7_h0000gn/T/ipykernel_18265/1438358905.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

[1] 44.694713  4.722982 66.258718
               [,1]      [,2]        [,3]     [,4]
sigmas -0.521757508 1.1841992 -2.54415596 2.333902
       -0.515816456 1.1752142 -2.53553442 2.375932
       -0.509910492 1.1662610 -2.52679644 2.418276
       -0.504039203 1.1573398 -2.51793909 2.460922
       -0.498202185 1.1484509 -2.50895954 2.503859
       -0.492399041 1.1395946 -2.49985505 2.547077
       -0.486629379 1.1307712 -2.49062295 2.590566
       -0.480892815 1.1219809 -2.48126067 2.634313
       -0.475188971 1.1132241 -2.47176573 2.678311
       -0.469517477 1.1045013 -2.46213574 2.722547
       -0.463877968 1.0958126 -2.45236837 2.767012
       -0.458270084 1.0871586 -2.44246139 2.811697
       -0.452693473 1.0785395 -2.43241266 2.856591
       -0.447147789 1.0699559 -2.42222008 2.901685
       -0.441632689 1.0614082 -2.41188168 2.946970
       -0.436147839 1.0528968 -2.40139553 2.992436
       -0.430692909 1.0444221 -2.39075979 3.038074
       -0.425267574 1.0359846 -2.37997268 3.0838

        0.141693132 0.4983319  0.63356757 7.564621
        0.144757603 0.5006666  0.65916863 7.544451
        0.147812711 0.5030880  0.68459588 7.522576
        0.150858515 0.5055953  0.70983931 7.498965
        0.153895069 0.5081873  0.73488876 7.473591
        0.156922431 0.5108631  0.75973390 7.446423
        0.159940656 0.5136214  0.78436422 7.417435
        0.162949798 0.5164612  0.80876911 7.386599
        0.165949913 0.5193811  0.83293780 7.353887
        0.168941053 0.5223798  0.85685939 7.319273
        0.171923274 0.5254561  0.88052286 7.282733
        0.174896627 0.5286085  0.90391711 7.244240
        0.177861166 0.5318356  0.92703089 7.203771
        0.180816942 0.5351359  0.94985291 7.161303
        0.183764008 0.5385078  0.97237177 7.116814
        0.186702413 0.5419497  0.99457602 7.070282
        0.189632210 0.5454600  1.01645416 7.021687
        0.192553448 0.5490368  1.03799465 6.971011
        0.195466177 0.5526786  1.05918591 6.918236
        0.198370447 0.5563833  

In [2]:
print(error_i)

[3, 4, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 37, 38, 42, 43, 47, 49, 50, 51, 55, 56, 60, 61, 64, 74, 78, 118, 135]
